In [ ]:
!pip install -q sentence-transformers faiss-cpu


In [ ]:
!pip install eventregistry

In [ ]:
import io
from sentence_transformers import SentenceTransformer, util
import numpy as np
import faiss
import textwrap
import json
import os

import string
from eventregistry import EventRegistry
import requests

In [ ]:
templates = [
    # =========================
    # PERFORMANCE / SENTIMENT
    # =========================
    {
        "id": 1,
        "code": "STRONG_BULL_STABLE",
        "category": "Performance / Sentiment",
        "use_when": "Revenue, earnings, and key metrics are consistently strong; volatility and risk appear limited.",
        "pattern": (
            "Company ___ demonstrates consistently strong and stable performance, supported by ___ [Sx]. "
            "Growth in ___ and solid margins in ___ underpin a positive outlook, with only limited near-term "
            "risks from ___ [Sy]."
        ),
    },
    {
        "id": 2,
        "code": "MODERATE_BULL_IMPROVING",
        "category": "Performance / Sentiment",
        "use_when": "Trends are improving (growth re-accelerating, margins recovering) but not spectacular; some risks remain.",
        "pattern": (
            "Company ___ shows improving performance, with ___ trending higher and ___ stabilizing [Sx]. "
            "While risks such as ___ remain, the overall trajectory appears positive over the recent period [Sy]."
        ),
    },
    {
        "id": 3,
        "code": "NEUTRAL_MIXED",
        "category": "Performance / Sentiment",
        "use_when": "Signals are mixed (some metrics up, others down; conflicting commentary).",
        "pattern": (
            "Company ___ presents a mixed picture. On one hand, ___ has improved or remained resilient [Sx], "
            "but on the other, ___ has weakened or introduces uncertainty [Sy]. Overall, the outlook is balanced "
            "with both upside and downside factors to monitor."
        ),
    },
    {
        "id": 4,
        "code": "MODERATE_BEAR_WEAKENING",
        "category": "Performance / Sentiment",
        "use_when": "Key metrics are weakening, but not catastrophic; there is still runway.",
        "pattern": (
            "Company ___ appears to be weakening, as evidenced by declines in ___ and pressure on ___ [Sx]. "
            "Management commentary and external news highlight concerns around ___ [Sy]. While not yet distressed, "
            "the trend warrants caution."
        ),
    },
    {
        "id": 5,
        "code": "STRONG_BEAR_DISTRESSED",
        "category": "Performance / Sentiment",
        "use_when": "Clear distress: sharp revenue/earnings declines, liquidity stress, credit or going-concern risks.",
        "pattern": (
            "Company ___ shows signs of financial distress. Sharp declines in ___ and mounting pressures on ___ "
            "indicate a deteriorating position [Sx]. News and filings cite serious risks such as ___, which "
            "significantly increase downside risk for investors [Sy]."
        ),
    },
    {
        "id": 6,
        "code": "TURNAROUND_RECOVERY",
        "category": "Performance / Sentiment",
        "use_when": "Past weakness but recent credible improvements (restructuring, margin recovery, debt reduction).",
        "pattern": (
            "Company ___ appears to be in a turnaround phase. After prior weakness in ___ [Sx], recent developments "
            "such as ___ and improvements in ___ indicate early signs of recovery [Sy]. The trajectory is improving, "
            "but execution risk remains."
        ),
    },

    # =========================
    # GROWTH / RISK BALANCE
    # =========================
    {
        "id": 7,
        "code": "HIGH_GROWTH_HIGH_RISK",
        "category": "Growth / Risk Balance",
        "use_when": "Very strong growth but with meaningful uncertainties (early-stage, customer concentration, regulatory risk).",
        "pattern": (
            "Company ___ is delivering high growth, with strong expansion in ___ and rapid adoption in ___ [Sx]. "
            "However, this comes with elevated risk from factors such as ___ and dependence on ___ [Sy]. "
            "The profile is best characterized as high-growth, high-risk."
        ),
    },
    {
        "id": 8,
        "code": "MATURE_STABLE_INCOME",
        "category": "Growth / Risk Balance",
        "use_when": "Low-to-moderate growth, stable cash flows, often dividends; defensive profile.",
        "pattern": (
            "Company ___ operates as a mature, relatively stable business. Revenue and cash flows from ___ are steady, "
            "with limited but predictable growth [Sx]. This profile may appeal to investors seeking income and stability "
            "rather than aggressive upside."
        ),
    },
    {
        "id": 9,
        "code": "CYCLICAL_EXPOSURE",
        "category": "Growth / Risk Balance",
        "use_when": "Performance is clearly tied to macro/sector cycle (commodities, housing, autos, etc.).",
        "pattern": (
            "Company ___ is strongly exposed to the economic cycle, particularly through its dependence on ___ [Sx]. "
            "Current results reflect the phase of the cycle, with ___ benefiting/hurting performance [Sy]. Future "
            "outcomes will largely track broader sector and macro conditions."
        ),
    },
    {
        "id": 10,
        "code": "EVENT_DRIVEN_MIXED",
        "category": "Growth / Risk Balance",
        "use_when": "M&A, litigation, restructuring, or one-off events dominate the outlook (positive or negative, but uncertain).",
        "pattern": (
            "Company ___ is currently driven by specific events, including ___ [Sx]. These developments could materially "
            "reshape its financial profile, but their ultimate impact remains uncertain. Investors should focus on "
            "milestones such as ___ and potential outcomes around ___ [Sy]."
        ),
    },

    # =====================================
    # RISK / BALANCE SHEET & GOVERNANCE
    # =====================================
    {
        "id": 11,
        "code": "LEVERAGE_LIQUIDITY_CONCERN",
        "category": "Risk / Balance Sheet & Governance",
        "use_when": "High leverage, tight liquidity, covenant/rollover risk show up in filings or news.",
        "pattern": (
            "Company ___ carries notable balance sheet risk. Elevated leverage in ___ and liquidity pressures from ___ "
            "increase vulnerability to adverse conditions [Sx]. Management and external commentary highlight concerns "
            "around ___, which investors should monitor closely [Sy]."
        ),
    },
    {
        "id": 12,
        "code": "REGULATORY_OR_LEGAL_RISK",
        "category": "Risk / Balance Sheet & Governance",
        "use_when": "Material regulatory investigations, legal actions, or policy changes could hit the business.",
        "pattern": (
            "Company ___ faces material regulatory or legal risk. Filings and news highlight issues related to ___ and "
            "potential impacts from ___ [Sx]. While the ultimate outcome is uncertain, these exposures could "
            "significantly affect profitability or valuation [Sy]."
        ),
    },
    {
        "id": 13,
        "code": "GOVERNANCE_OR_EXECUTION_RISK",
        "category": "Risk / Balance Sheet & Governance",
        "use_when": "Management credibility, execution on strategy, or governance structures are flagged.",
        "pattern": (
            "For Company ___, governance and execution are key risks. Sources point to concerns around ___, management "
            "decisions on ___, or challenges delivering on ___ [Sx]. These factors may weigh on investor confidence "
            "even if the core fundamentals remain ___ [Sy]."
        ),
    },

    # =========================
    # DATA QUALITY / UNCERTAINTY
    # =========================
    {
        "id": 14,
        "code": "DATA_CONFLICTING_EVIDENCE",
        "category": "Data Quality / Uncertainty",
        "use_when": "Filings, news, or metrics disagree; you can’t form a clear directional call.",
        "pattern": (
            "The available evidence for Company ___ is conflicting. While some snippets indicate ___ [Sx], others "
            "highlight opposing signals such as ___ [Sy]. Given these inconsistencies, any conclusion should be "
            "treated with caution and updated as new information emerges."
        ),
    },
    # {
    #     "id": 15,
    #     "code": "DATA_INSUFFICIENT_OR_OUTDATED",
    #     "category": "Data Quality / Uncertainty",
    #     "use_when": "Very little relevant or recent info retrieved; you honestly don’t know.",
    #     "pattern": (
    #         "There is insufficient recent information to form a confident view on Company ___. The retrieved snippets "
    #         "provide only limited or outdated detail on key metrics such as ___ [Sx]. A more reliable assessment "
    #         "would require fresher filings or news coverage."
    #     ),
    # },

    # =====================================
    # RISK FACTORS & EXPOSURES (SINGLE CO.)
    # =====================================
    {
        "id": 15,
        "code": "RISK_FACTORS_OVERVIEW",
        "category": "Risk Factors & Exposures",
        "use_when": "User asks about main risk factors for a single company; multiple identifiable risk themes.",
        "pattern": (
            "Key risk factors for Company ___ include ___, ___, and ___ [Sx]. These relate to areas such as demand "
            "sensitivity in ___, operational or execution risk in ___, and financial or balance sheet exposure from "
            "___ [Sy]. Taken together, they could affect growth, margins, or valuation if conditions worsen."
        ),
    },
    {
        "id": 16,
        "code": "MACRO_SECTOR_RISK_EXPOSURE",
        "category": "Risk Factors & Exposures",
        "use_when": "Company is highly exposed to macro variables or sector-wide conditions (rates, FX, commodities, housing cycle, etc.).",
        "pattern": (
            "Company ___ is meaningfully exposed to macro and sector conditions. Performance is sensitive to "
            "developments in ___ (e.g., interest rates, commodity prices, or end-market demand) [Sx]. Recent "
            "commentary highlights that changes in ___ and broader sector trends in ___ are key swing factors for "
            "future results [Sy]."
        ),
    },
    {
        "id": 17,
        "code": "COMPETITIVE_INTENSITY_RISK",
        "category": "Risk Factors & Exposures",
        "use_when": "Competition, pricing pressure, or loss of share shows up as a key risk.",
        "pattern": (
            "For Company ___, competitive intensity is a notable risk. Sources point to pressure from rivals in ___, "
            "including pricing pressure in ___ and potential share loss in ___ [Sx]. If competition continues to "
            "intensify or new entrants emerge, this could weigh on revenue growth and margins [Sy]."
        ),
    },
    {
        "id": 18,
        "code": "CONCENTRATION_DEPENDENCE_RISK",
        "category": "Risk Factors & Exposures",
        "use_when": "Business depends heavily on a small number of customers, suppliers, products, or geographies.",
        "pattern": (
            "Company ___ faces concentration risk, with a significant dependence on ___ (e.g., a few key customers, "
            "products, or regions) [Sx]. Adverse changes such as contract losses, regulatory shifts in ___, or "
            "disruption at key suppliers could disproportionately impact results [Sy]. Diversification remains a "
            "medium-term mitigation focus."
        ),
    },
    {
        "id": 19,
        "code": "RISK_MITIGANTS_AND_CATALYSTS",
        "category": "Risk Factors & Exposures",
        "use_when": "User asks how risks might be mitigated or what could improve the situation; clear management actions or catalysts exist.",
        "pattern": (
            "While Company ___ faces risks around ___ and ___ [Sx], there are also mitigating factors and potential "
            "catalysts. Management is pursuing measures such as ___ and ___, and upcoming events like ___ "
            "(e.g., product launches, regulatory decisions, or debt refinancings) could reduce risk or unlock upside [Sy]."
        ),
    },

    # =========================================
    # PEER COMPARISON & RELATIVE POSITIONING
    # =========================================
    {
        "id": 20,
        "code": "PEER_COMPARISON_A_STRONGER",
        "category": "Peer Comparison & Relative Positioning",
        "use_when": "Evidence suggests Company A is clearly stronger overall than Company B on fundamentals or risk profile.",
        "pattern": (
            "Compared with Company ___ (B), Company ___ (A) appears stronger overall. A shows better performance in "
            "___ (e.g., growth, margins, or cash generation) and a more resilient balance sheet or risk profile in "
            "___ [Sx]. While B offers some strengths in ___, its exposure to risks such as ___ makes it relatively "
            "less attractive on a risk-adjusted basis [Sy]."
        ),
    },
    {
        "id": 21,
        "code": "PEER_COMPARISON_B_STRONGER",
        "category": "Peer Comparison & Relative Positioning",
        "use_when": "Evidence suggests Company B is clearly stronger than Company A.",
        "pattern": (
            "Relative to Company ___ (B), Company ___ (A) looks weaker on several dimensions. A lags in areas such as "
            "___ (growth, profitability, or scale) and faces higher risk from ___ [Sx]. By contrast, B benefits from "
            "stronger positioning in ___ and more manageable exposure to ___, suggesting a more favorable overall "
            "profile at present [Sy]."
        ),
    },
    {
        "id": 22,
        "code": "PEER_COMPARISON_MIXED_TRADEOFFS",
        "category": "Peer Comparison & Relative Positioning",
        "use_when": "Both companies have meaningful pros/cons; user asks how A compares to B or which is better, but answer is nuanced.",
        "pattern": (
            "Company ___ (A) and Company ___ (B) present a mixed comparison. A offers strengths in ___ "
            "(e.g., higher growth, stronger innovation, or specific segment leadership) [Sx], while B stands out in "
            "___ (e.g., margin stability, balance sheet strength, or lower cyclicality) [Sy]. The more attractive "
            "choice depends on whether the priority is upside potential (A) or stability and risk control (B)."
        ),
    },
    {
        "id": 23,
        "code": "PEER_COMPARISON_RISK_FOCUSED",
        "category": "Peer Comparison & Relative Positioning",
        "use_when": "Question is which of A or B is riskier, or how their risks compare.",
        "pattern": (
            "From a risk perspective, Company ___ faces greater exposure to ___ (e.g., leverage, regulatory, or "
            "competitive risks), whereas Company ___ is more exposed to ___ [Sx]. A’s risk profile is driven by "
            "factors such as ___ and sensitivity to ___, while B’s key vulnerabilities relate to ___ [Sy]. Overall, "
            "___ appears to carry higher downside risk, while ___ may be relatively more defensive, assuming current "
            "conditions persist."
        ),
    },

    # =========================================
    # “SHOULD I INVEST?” / ALLOCATION FRAMING
    # =========================================
    {
        "id": 24,
        "code": "INVESTMENT_VIEW_A_PREFERRED",
        "category": "Investment View / Allocation",
        "use_when": "User asks whether to invest in A or B and evidence leans toward A on a risk-reward basis.",
        "pattern": (
            "Based on the available information, Company ___ (A) currently looks more attractive than Company ___ (B) "
            "on a risk-reward basis. A offers advantages such as ___ (e.g., stronger growth trajectory, healthier "
            "balance sheet, clearer catalysts) [Sx], while B is weighed down by risks around ___ [Sy]. This is a "
            "general analytical view, not personalized investment advice; suitability still depends on your "
            "objectives, risk tolerance, and portfolio context."
        ),
    },
    {
        "id": 25,
        "code": "INVESTMENT_VIEW_B_PREFERRED",
        "category": "Investment View / Allocation",
        "use_when": "User asks whether to invest in A or B and evidence leans toward B.",
        "pattern": (
            "Given the current data, Company ___ (B) appears more compelling than Company ___ (A) from a risk-reward "
            "perspective. B benefits from strengths in ___ and more manageable exposure to ___ [Sx], whereas A faces "
            "headwinds from ___ or greater uncertainty around ___ [Sy]. This should be treated as a high-level "
            "assessment rather than individualized investment advice."
        ),
    },
    {
        "id": 26,
        "code": "INVESTMENT_VIEW_DEPENDS_ON_PROFILE",
        "category": "Investment View / Allocation",
        "use_when": "User asks about investing in a single company or choosing between A and B; correct framing is that it depends on risk/return preferences.",
        "pattern": (
            "Whether Company ___ is suitable as an investment depends heavily on your risk tolerance and goals. "
            "The company offers potential upside from ___ and ___ [Sx], but also carries risks related to ___ and "
            "___ [Sy]. Investors seeking higher growth and willing to accept volatility may view the profile more "
            "positively, while more conservative investors might prefer companies with steadier cash flows and fewer "
            "specific risk factors. This is general information, not personalized investment advice."
        ),
    },
    {
        "id": 27,
        "code": "INVESTMENT_VIEW_TOO_UNCERTAIN",
        "category": "Investment View / Allocation",
        "use_when": "User asks 'Should I invest?' but data is conflicting/limited or risks are too unclear for a confident view.",
        "pattern": (
            "It is difficult to form a clear investment view on Company ___ based on the current information. "
            "While there are potential positives such as ___ [Sx], there are also material uncertainties or data gaps "
            "around ___ [Sy]. Given this uncertainty, any investment decision would require additional, up-to-date "
            "analysis of the company’s financials, competitive position, and your own circumstances."
        ),
    },

]

# templates to be used later

# special_templates = [
#     {
#         "id": 29,
#         "code": "LIMITED_INFORMATION_RESPONSE",
#         "category": "Out of Scope / Limited Information",
#         "use_when": "Available information does not clearly answer the user’s question about the company.",
#         "pattern": (
#             "The available information does not clearly answer this question about Company ___. The sources mainly "
#             "discuss ___ and ___, but do not provide reliable detail on ___ [Sx]. Any conclusion would be speculative; "
#             "instead, we can say that current disclosures highlight ___ and leave uncertainty around ___ [Sy]."
#         ),
#     },
# ]

In [ ]:
# build texts and metadata analogous to Naz's code
# treating pattern as actual text
# remaining fields as metadata

template_texts = []
template_meta = []

for t in templates:
    template_texts.append(t["pattern"])
    template_meta.append({
        "id": t["id"],
        "code": t["code"],
        "category": t["category"],
        "use_when": t["use_when"]
    })

print("Number of templates:", len(template_texts))
print("Example meta:", template_meta[0])
print("Example text:", template_texts[0])

Number of templates: 27
Example meta: {'id': 1, 'code': 'STRONG_BULL_STABLE', 'category': 'Performance / Sentiment', 'use_when': 'Revenue, earnings, and key metrics are consistently strong; volatility and risk appear limited.'}
Example text: Company ___ demonstrates consistently strong and stable performance, supported by ___ [Sx]. Growth in ___ and solid margins in ___ underpin a positive outlook, with only limited near-term risks from ___ [Sy].


In [ ]:
# defining embedding model
model_name = "BAAI/bge-small-en"
embed_model = SentenceTransformer(model_name)
print("Model loaded:", model_name)

# create embeddings for all template texts
template_embeddings = embed_model.encode(
    template_texts,
    batch_size=16,
    show_progress_bar=True
).astype("float32")

num_templates, dim = template_embeddings.shape
print("Embeddings shape:", template_embeddings.shape)

Model loaded: BAAI/bge-small-en


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Embeddings shape: (27, 384)


In [ ]:
# create FAISS index for template embeddings
dim = template_embeddings.shape[1]
index = faiss.IndexFlatIP(dim)

# normalize vectors for cosine similarity
faiss.normalize_L2(template_embeddings)
index.add(template_embeddings)

print("FAISS index size:", index.ntotal)

FAISS index size: 27


In [ ]:
# used chatGPT to add verbose condition for stopping duplicate prints

def search_templates(query, top_k=5, verbose=True):
    """
    Return the top-k matching template patterns based on user query.
    """
    # Embed the query itself
    q_emb = embed_model.encode([query]).astype("float32")
    faiss.normalize_L2(q_emb)

    # Search FAISS db
    distances, indices = index.search(q_emb, top_k)

    results = []

    if verbose:
        print(f"\nQuery: {query}\n")

    for rank, (score, idx) in enumerate(zip(distances[0], indices[0]), start=1):
        meta = template_meta[idx]
        text = template_texts[idx]

        if verbose:
            print(f"── Result {rank} ──")
            print(f"Code: {meta['code']} | Category: {meta['category']} | id={meta['id']}")
            print("Template:")
            print(textwrap.fill(text, width=100))
            print()

        results.append({
            "rank": rank,
            "id": meta["id"],
            "code": meta["code"],
            "category": meta["category"],
            "use_when": meta["use_when"],
            "pattern": text,
        })

    return results

In [ ]:
# testing

_ = search_templates("How is this company's financial performance?", top_k=5)
_ = search_templates("What are the key risks for this business?", top_k=5)
_ = search_templates("Should I invest in Intel or Nvidia?", top_k=5)
_ = search_templates("Give me the outlook and guidance", top_k=5)


Query: How is this company's financial performance?

── Result 1 ──
Code: MATURE_STABLE_INCOME | Category: Growth / Risk Balance | id=8
Template:
Company ___ operates as a mature, relatively stable business. Revenue and cash flows from ___ are
steady, with limited but predictable growth [Sx]. This profile may appeal to investors seeking
income and stability rather than aggressive upside.

── Result 2 ──
Code: TURNAROUND_RECOVERY | Category: Performance / Sentiment | id=6
Template:
Company ___ appears to be in a turnaround phase. After prior weakness in ___ [Sx], recent
developments such as ___ and improvements in ___ indicate early signs of recovery [Sy]. The
trajectory is improving, but execution risk remains.

── Result 3 ──
Code: NEUTRAL_MIXED | Category: Performance / Sentiment | id=3
Template:
Company ___ presents a mixed picture. On one hand, ___ has improved or remained resilient [Sx], but
on the other, ___ has weakened or introduces uncertainty [Sy]. Overall, the outlook is ba

In [ ]:
def get_candidate_template(user_question: str, top_k: int = 5):
    """
    return the raw list of templates
    from search_templates.
    """
    raw_results = search_templates(user_question, top_k=top_k, verbose=False)

    cleaned = []
    for r in raw_results:
        cleaned.append({
            "code": r["code"],
            "use_when": r["use_when"],
            "pattern": r["pattern"]
        })

    return cleaned

In [ ]:
print(get_candidate_template("How is this company's financial performance?"))

[{'code': 'MATURE_STABLE_INCOME', 'use_when': 'Low-to-moderate growth, stable cash flows, often dividends; defensive profile.', 'pattern': 'Company ___ operates as a mature, relatively stable business. Revenue and cash flows from ___ are steady, with limited but predictable growth [Sx]. This profile may appeal to investors seeking income and stability rather than aggressive upside.'}, {'code': 'TURNAROUND_RECOVERY', 'use_when': 'Past weakness but recent credible improvements (restructuring, margin recovery, debt reduction).', 'pattern': 'Company ___ appears to be in a turnaround phase. After prior weakness in ___ [Sx], recent developments such as ___ and improvements in ___ indicate early signs of recovery [Sy]. The trajectory is improving, but execution risk remains.'}, {'code': 'NEUTRAL_MIXED', 'use_when': 'Signals are mixed (some metrics up, others down; conflicting commentary).', 'pattern': 'Company ___ presents a mixed picture. On one hand, ___ has improved or remained resilient [

# Helper Function For Fin Docs (already chunked)

In [ ]:
from google.colab import files

# chunks.json
# chunk_meta.json
# chunk_embeddings.npy
# faiss_index.bin

uploaded = files.upload()
print("Uploaded files:", list(uploaded.keys()))

Saving faiss.index to faiss (5).index
Saving chunk_embeddings.npy to chunk_embeddings (5).npy
Saving chunk_meta.json to chunk_meta (5).json
Saving chunks.json to chunks (5).json
Uploaded files: ['faiss (5).index', 'chunk_embeddings (5).npy', 'chunk_meta (5).json', 'chunks (5).json']


In [ ]:
# loading pre-saved chunks
# metadata + raw text
with open("chunks.json", "r", encoding="utf-8") as f:
    chunks = json.load(f)

with open("chunk_meta.json", "r", encoding="utf-8") as f:
    chunk_meta = json.load(f)

# FAISS index
index_vals = faiss.read_index("faiss.index")

# embedding model
model_name = "BAAI/bge-small-en"
embed_model = SentenceTransformer(model_name)
print("Loaded model and index. Num chunks:", len(chunks))

Loaded model and index. Num chunks: 5454


In [ ]:
def retrieve_snippets(query: str, top_k: int = 5):
    """
    Given a user query, return top_k relevant chunks along with metadata.
    """

    # embed query
    q_emb = embed_model.encode([query]).astype("float32")
    faiss.normalize_L2(q_emb)

    # search FAISS index
    distances, indices = index_vals.search(q_emb, top_k)

    results = []
    for rank, (score, idx) in enumerate(zip(distances[0], indices[0]), start=1):
        if idx == -1:
            continue

        meta = chunk_meta[idx]
        text = chunks[idx]

        results.append({   # <-- MUST be inside the for-loop
            "snippet_id": f"S{rank}",
            "score": float(score),
            "filename": meta.get("filename") or meta.get("file"),
            "page_number": meta.get("page_number"),
            "chunk_in_page": meta.get("chunk_in_page"),
            "char_start": meta.get("char_start"),
            "char_end": meta.get("char_end"),
            "text": text,
        })

    return results

In [ ]:
# helper function - pass to llm later

#def snippet_for_prompt(snippets):
    #lines = ["Each snippet includes an ID and its source.\n"]

    #for s in snippets:
        #text = (s.get("text") or "").replace("\n", " ")
        #if len(text) > 500:
            #text = text[:500] + "..."

        # pull the right metadata keys
        # Include metadata in the snippet header
        # will be used as ref citations
        #filename = s.get("filename") or s.get("file") or "Unknown"
        #page = s.get("page_number")
        #chunk_in_page = s.get("chunk_in_page")

        # build a clean source label
        #src_bits = [f"file={filename}"]
        #if page is not None:
            #src_bits.append(f"page={page}")
        #if chunk_in_page is not None:
            #src_bits.append(f"chunk_in_page={chunk_in_page}")

        #source_str = " | ".join(src_bits)

        #lines.append(
            #f"[{s['snippet_id']}] ({source_str})\n"
            #f"{text}\n"
        #)

    #return "\n".join(lines)
import re

def normalize_ws(s: str) -> str:
    s = s.replace("\u00a0", " ")          # nbsp
    s = re.sub(r"[ \t]+", " ", s)         # collapse spaces/tabs
    s = re.sub(r"\n{3,}", "\n\n", s)      # cap runaway newlines
    return s.strip()

def snippet_for_prompt(snippets):
    lines = ["Each snippet includes an ID and its source.\n"]

    for s in snippets:
        text = normalize_ws(s["text"].replace("\n", " "))
        if len(text) > 500:
            text = text[:500] + "..."

        # Only expose meaningful metadata
        # will be used to ref citations
        source_parts = []
        if s.get("filename"):
            source_parts.append(f"file={s['filename']}")
        if s.get("page_number") is not None:
            source_parts.append(f"page={s['page_number']}")

        source_str = " | ".join(source_parts)

        lines.append(
            f"[{s['snippet_id']}] ({source_str})\n"
            f"{text}\n"
        )

    return "\n".join(lines)

In [ ]:
print("faiss ntotal:", index_vals.ntotal)
print("len(chunks):", len(chunks))
print("len(chunk_meta):", len(chunk_meta))

faiss ntotal: 5454
len(chunks): 5454
len(chunk_meta): 5454


In [ ]:
q = "What is revenue growth in NVIDIA?"
snips = retrieve_snippets(q, top_k=5)

print("Got", len(snips), "snippets\n")
for s in snips:
    print(f"--- {s['snippet_id']}")
    print(f"From file: {s.get('filename') or s.get('file')}")
    print(s['text'][:400], "...\n")

# See what the prompt block would look like
print(snippet_for_prompt(snips))

Got 5 snippets

--- S1
From file: NVIDIA-10Q_JUl25.pdf
0,208 $ 48,835
Compute 33,844 22,604 67,999 41,996
Networking 7,252 3,668 12,209 6,839
Gaming 4,287 2,880 8,050 5,527
Professional Visualization 601 454 1,110 881
Automotive 586 346 1,153 675
OEM and Other 173 88 284 166
Total revenue $ 46,743 $ 30,040 $ 90,805 $ 56,084
22 ...

--- S2
From file: NVIDIA-10K_2025JAN.pdf
age models, recommendation engines, and
generative AI applications. Revenue from Data Center networking grew 51% driven by Ethernet for AI revenue, which includes Spectrum-X end-to-end ethernet
platform.
Graphics revenue – The year over year increase was driven by sales of our GeForce RTX 40 Series GPUs.
Reportable segment operating income – The year over year increase in Compute & Networking segm ...

--- S3
From file: NVIDIA-10K_2025JAN.pdf
Table of Contents
Reportable Segments
Revenue by Reportable Segments
Year Ended
$ %
Jan 26, 2025 Jan 28, 2024 Change Change
($ in millions)
Compute & Networking $ 116,193 $ 47,40

# NewsAPI - Fetching Relevant News

In [ ]:
# extracting keywords from user query

# replace later with Naz's recommendation
news_embed_model = SentenceTransformer("all-MiniLM-L6-v2")

# Domain + finance keyword candidates
relevant_base_keywords = [
    # financial concepts
    "revenue growth",
    "earnings",
    "profitability",
    "operating margin",
    "guidance",
    "forecast",
    "outlook",
    "cash flow",
    "capital expenditure",
    "debt",
    "liquidity",
    "dividends",
    "share buyback",
    "valuation",

    # risk
    "competition",
    "competitive landscape",
    "emerging rivals",
    "market share",
    "regulatory risk",
    "macroeconomic risk",
    "geopolitical risk",
    "supply chain issues",

    # tech
    "semiconductor",
    "AI chips",
    "data center",
    "cloud computing",
    "smartphones",
    "memory chips",
    "foundry business",
]

# precomputing base keyword embeddings
base_key_embed = news_embed_model.encode(relevant_base_keywords, convert_to_tensor=True)


In [ ]:
# ======================================================================
# List of companies in corpus with aliases - used ChatGPT
# ======================================================================

supported_companies = [
    {
        "name": "NVIDIA",
        "aliases": ["nvidia", "nvda", "nvidia corp", "nvidia corporation"],
    },
    {
        "name": "Intel",
        "aliases": ["intel", "intc", "intel corp", "intel corporation"],
    },
    {
        "name": "Samsung Electronics",
        "aliases": ["samsung", "samsung electronics", "samsung elec"],
    },
    {
        "name": "Taiwan Semiconductor",
        "aliases": ["tsmc", "taiwan semiconductor", "taiwan semi"],
    },
]

# Flatten aliases into a list + lookup back to company names
company_alias_texts = []
company_alias_to_company = []

for c in supported_companies:
    for alias in c["aliases"]:
        company_alias_texts.append(alias.lower())
        company_alias_to_company.append(c["name"])

# compute embeddings for all aliases
company_alias_embeds = news_embed_model.encode(company_alias_texts, convert_to_tensor=True)



In [ ]:
def clean_query_format(text: str) -> str:
    # replace punctuation with spaces
    table = str.maketrans({c: " " for c in string.punctuation})
    return text.lower().translate(table)



def detect_company_from_query(user_query: str, min_sim: float = 0.35):
    """
    Use embeddings over the small alias list to guess which supported company
    the user is asking about. Returns company name or None.
    """
    cleaned = clean_query_format(user_query)

    q_emb = news_embed_model.encode(cleaned, convert_to_tensor=True)
    sims = util.cos_sim(q_emb, company_alias_embeds)[0]

    # Get best match
    best_score, best_idx = sims.max(dim=0)
    best_score = float(best_score)
    best_idx = int(best_idx)

    if best_score < min_sim:
        # no confident match
        return None

    company_name = company_alias_to_company[best_idx]
    return company_name



# used chatGPT for min_sim and cos_sim
def build_keywords_string(
    user_query: str,
    company: str = None,
    max_terms: int = 3,
    min_sim: float = 0.35, # keeping similarity threshold on lower end
) -> str:
    """
    Use embeddings to find relevant keywords from relevant_base_keywords
    for a user query to build a string of keywords that is later passed onto
    the newsAPI.
    """

    if company is None:
        company = detect_company_from_query(user_query)

    cleaned = clean_query_format(user_query)

    # embed the user full query
    query_emb = news_embed_model.encode(cleaned, convert_to_tensor=True)

    # compute similarity to base keywords
    sims = util.cos_sim(query_emb, base_key_embed)[0]  # vector of cosine similarity scores

    # used chatGPT for generating code to get top indices by similarity
    k = min(max_terms, len(relevant_base_keywords))
    top_vals, top_idxs = sims.topk(k=k)

    chosen_terms = []
    for score, idx in zip(top_vals.tolist(), top_idxs.tolist()):
        if score >= min_sim:
            term = relevant_base_keywords[idx]
            chosen_terms.append(term)

    # build final keyword string
    # Keyword string format: start with company name then join terms with spaces
    parts = [company] + chosen_terms
    keyword_str = " ".join(parts)

    return keyword_str


In [ ]:
# pulling relevant news articles from NewsAPI

"""Documentation for using NewsAPI: https://newsapi.ai/documentation?tab=searchArticles&lang=rest"""

API_KEY = "f8e1482c-c5cf-4f48-9049-31046857b5a9"

er = EventRegistry(apiKey="f8e1482c-c5cf-4f48-9049-31046857b5a9")


news_sources = [
    "Bloomberg",
    "Reuters",
    "Financial Times",
    "Wall Street Journal",
    "Yahoo Finance",
    "CNBC",
    "MarketWatch",
    "Business Insider",
    "The Economist",
    "New York Times",
    "Washington Post",
    "BBC",
    "The Guardian",
    "AP News",
    "NPR"
]

source_uris = {}

for name in news_sources:
    uri = er.getSourceUri(name)
    source_uris[name] = uri
    # print(f"{name}: {uri}")


def news_request(user_query: str,
                            company: str = None,
                            articles_count: int = 3,
                            days_window: int = 7) -> dict:
    """
    hit newsAPI endpoints to extract relevant articles
    """

    # build keyword string
    keyword_str = build_keywords_string(
        user_query=user_query,
        company=company,
        max_terms=3,     # max no of words passed on as keyword category to newsAPI
        min_sim=0.35,
    )

    # use only relevant news sources
    sources_list = list(source_uris.values())

    # 3) request body - filters retrieved from NewsAPI documentation
    body = {
        "apiKey": API_KEY,
        "keyword": keyword_str,
        "lang": "eng",
        "sourceUri": sources_list,
        #"articlesPage": 1,
        "articlesCount": articles_count,
        "articlesSortBy": "rel", # sort articles by relevance to query
        "articlesSortByAsc": False,
        "dataType": ["news"],
        "isDuplicateFilter": "skipDuplicates",
        "forceMaxDataTimeWindow": days_window,
        "includeArticleBody": True,
        #"articleBodyLen": -1,
        "includeArticleSentiment": True,
    }
    return body

def interpret_sentiment(score: float) -> str:
    if score is None:
        return "Unknown sentiment"

    if score > 0.35:
        return f"Positive ({score:.2f})"
    elif score > 0.1:
        return f"Slightly Positive ({score:.2f})"
    elif score >= -0.1:
        return f"Neutral ({score:.2f})"
    elif score >= -0.35:
        return f"Slightly Negative ({score:.2f})"
    else:
        return f"Negative ({score:.2f})"


def fetch_news(user_query: str,
               company: str = None,
               articles_count: int = 3,
               days_window: int = 7):
    """
    response function that pulls relevant articles from NewsAPI based on the following conditions:
    - keywords extracted from user query
    - request body filters
    """
    body = news_request(
        user_query=user_query,
        company=company,
        articles_count=articles_count,
        days_window=days_window,
    )

    response = requests.post(
        "https://eventregistry.org/api/v1/article/getArticles",
        headers={"Content-Type": "application/json"},
        data=json.dumps(body))

    data = response.json()
    raw_articles = data.get("articles", {}).get("results", [])


    # used chatGPT for response formatting
    cleaned = []
    for art in raw_articles:
        cleaned.append({
            "title":     art.get("title"),
            "source":    art.get("source", {}).get("title"),
            "date":      art.get("dateTimePub"),
            "url":       art.get("url"),
            "sentiment": interpret_sentiment(art.get("sentiment")),
            "body": art.get("body")
        })

    return cleaned


In [ ]:
# function to retrieve and ingest news articles to prompt
def news_for_prompt(
    articles: list,
    max_articles: int = 3,
) -> str:
    """
    Format cleaned news articles for the LLM prompt.

    Expects each article dict to have:
      - title
      - source
      - date
      - url
      - sentiment
    """
    if not articles:
        return "[RECENT NEWS]\nNo relevant news articles within the past 7 days were retrieved.\n"

    lines = []
    lines.append("[RECENT NEWS]")
    lines.append("Below are recent news articles related to the company.")
    lines.append("You can use them as supplementary context for sentiment, risks, and catalysts.\n")

    for i, art in enumerate(articles[:max_articles], start=1):
        body = (art.get("body") or "").replace("\n", " ")
        body_preview = body[:300] + ("..." if len(body) > 300 else "")

        lines.append(f"[NEWS{i}] {art.get('title') or 'Untitled'}")
        lines.append(
            f"Source: {art.get('source') or 'Unknown'} | "
            f"Date: {art.get('date') or 'Unknown'} | "
            f"Sentiment: {art.get('sentiment') or 'Unknown'}"
        )
        lines.append(f"URL: {art.get('url') or 'N/A'}")
        lines.append(f"Article preview: {body_preview}")
        lines.append("")  # blank line between articles

    return "\n".join(lines)



In [ ]:

# testing
news_q = "How is Nvidia performing lately?"
articles = fetch_news(news_q, articles_count=3, days_window=7)
# print(json.dumps(articles, indent=2))

news_block = news_for_prompt(articles, max_articles=3)
print(news_block)

[RECENT NEWS]
Below are recent news articles related to the company.
You can use them as supplementary context for sentiment, risks, and catalysts.

[NEWS1] Transcript: Business Book of the Year  --  Author Stephen Witt on Nvidia's rise
Source: Financial Times News | Date: 2025-12-11T07:02:16Z | Sentiment: Slightly Positive (0.25)
URL: https://www.ft.com/content/08c8b9de-cf91-4c7c-a10d-cf15a09af48a
Article preview: Michela Tindera  Hey there, Behind the Money listeners. It's Michela. Last week, the Financial Times and Schroders gave out their prize for the best business book of the year. And the winner was a book about the company that has dominated headlines this year, Nvidia. The book is called The Thinking ...

[NEWS2] Nvidia's bumpy November
Source: Business Insider | Date: 2025-11-27T09:30:01Z | Sentiment: Slightly Positive (0.15)
URL: https://www.businessinsider.com/nvidia-november-challenges-google-michael-burry-ai-bubble-fears-2025-11
Article preview: News of a potential big Go

# LLM Template (Putting everything together)

In [ ]:
def templates_for_prompt(templates):
    """
    Turn a list of template dicts into the [CANDIDATE TEMPLATES] text block.
    """
    lines = []
    lines.append("[CANDIDATE TEMPLATES]\n")
    lines.append("You must choose exactly ONE of the candidate templates below.\n")

    for i, t in enumerate(templates, start=1):
        lines.append(
            f"\n[T{i}] TEMPLATE_ID: {t['code']}\n"
            f"Use when: {t['use_when']}\n"
            f"Pattern: {t['pattern']}\n"
        )

    return "\n".join(lines)

In [ ]:
test = get_candidate_template("How is this company's performance?")
print(templates_for_prompt(test))

[CANDIDATE TEMPLATES]

You must choose exactly ONE of the candidate templates below.


[T1] TEMPLATE_ID: MATURE_STABLE_INCOME
Use when: Low-to-moderate growth, stable cash flows, often dividends; defensive profile.
Pattern: Company ___ operates as a mature, relatively stable business. Revenue and cash flows from ___ are steady, with limited but predictable growth [Sx]. This profile may appeal to investors seeking income and stability rather than aggressive upside.


[T2] TEMPLATE_ID: TURNAROUND_RECOVERY
Use when: Past weakness but recent credible improvements (restructuring, margin recovery, debt reduction).
Pattern: Company ___ appears to be in a turnaround phase. After prior weakness in ___ [Sx], recent developments such as ___ and improvements in ___ indicate early signs of recovery [Sy]. The trajectory is improving, but execution risk remains.


[T3] TEMPLATE_ID: MODERATE_BULL_IMPROVING
Use when: Trends are improving (growth re-accelerating, margins recovering) but not spectacular;

In [ ]:
def build_llm_prompt(
    user_question: str,
    top_k_snippets: int = 5,
    top_k_templates: int = 5,
    use_news: bool = True,
    news_articles: list[dict] | None = None,
    news_max_articles: int = 3,
    news_days_window: int = 7,
) -> dict:
    """
    Building the full prompt for the LLM plus returning the snippets and templates.
    """
    # retrieve financial docs snippets from vector store
    snippets = retrieve_snippets(user_question, top_k=top_k_snippets)

    # retrieve candidate templates from vector store
    candidate_templates = get_candidate_template(user_question, top_k=top_k_templates)

    if news_articles is None and use_news:
      news_articles = fetch_news(
          user_query=user_question,
          articles_count=news_max_articles,
          days_window=news_days_window,
      )

    # system prompt
    system_prompt = """[SYSTEM]
You are a cautious financial analysis assistant.
You must:
- Rely ONLY on the snippets provided as evidence.
- Be explicit about uncertainty.
- Avoid making up numbers or facts that are not in the snippets.
- Briefly explain your reasoning and cite snippet IDs like [S1], [S2].
"""

    # user question block
    user_block = f"[USER QUESTION]\n{user_question}\n"

    # snippets block
    snippets_block = snippet_for_prompt(snippets)

    # templates block
    templates_block = templates_for_prompt(candidate_templates)

    # news block
    news_block = ""
    if news_articles:
      news_block = news_for_prompt(news_articles, max_articles=news_max_articles)

    # 7) instructions block
    instructions_block = """[INSTRUCTIONS]
1. Decide which single TEMPLATE_ID from the candidates best matches the situation.
2. Produce a short, professional answer (2–5 sentences) using that pattern.
3. Use the snippets as evidence and cite them using [S1], [S2], etc.
4. If evidence is sparse or conflicting, say so explicitly.

Respond in the following STRICT format:

TEMPLATE_ID: <one of the candidate template IDs, e.g., STRONG_BULL_STABLE>

ANSWER:
<your final answer here with citations, 2–5 sentences>
"""

    # assemble prompt
    blocks = [
        system_prompt,
        user_block,
        snippets_block,
    ]

    if news_block:
      blocks.append(news_block)

    blocks.extend([
        templates_block,
        instructions_block,
    ])


    full_prompt = "\n\n".join(blocks)

    return {
        "prompt": full_prompt,
        "snippets": snippets,
        "templates": templates_block,
        "news_articles": news_articles,
    }

In [ ]:
test_q = "How is Nvidia performing lately?"
result = build_llm_prompt(test_q)

print(result["prompt"])  # print first ~2000 chars so it doesn't explode

[SYSTEM]
You are a cautious financial analysis assistant.
You must:
- Rely ONLY on the snippets provided as evidence.
- Be explicit about uncertainty.
- Avoid making up numbers or facts that are not in the snippets.
- Briefly explain your reasoning and cite snippet IDs like [S1], [S2].


[USER QUESTION]
How is Nvidia performing lately?


Each snippet includes an ID and its source.

[S1] (file=NVIDIA-10Q_JUl25.pdf | page=22)
0,208 $ 48,835 Compute 33,844 22,604 67,999 41,996 Networking 7,252 3,668 12,209 6,839 Gaming 4,287 2,880 8,050 5,527 Professional Visualization 601 454 1,110 881 Automotive 586 346 1,153 675 OEM and Other 173 88 284 166 Total revenue $ 46,743 $ 30,040 $ 90,805 $ 56,084 22

[S2] (file=NVIDIA-10Q_JUl25.pdf | page=42)
NVIDIA Corporation Participant: By: Signature Signature Title: Date: Date:

[S3] (file=NVIDIA-10Q_JUl25.pdf | page=33)
rienced and may continue to experience challenges including lengthy and costly systems integration, delays in purchasing and shipping p

In [ ]:
def updated_build_llm_prompt(
    user_question: str,
    top_k_snippets: int = 7,
    top_k_templates: int = 6,
    use_news: bool = True,
    news_articles: list[dict] | None = None,
    news_max_articles: int = 3,
    news_days_window: int = 7,
) -> dict:
    """
    Building the full prompt for the LLM plus returning the snippets and templates. Using updated prompt.
    """
    # retrieve financial docs snippets from vector store
    snippets = retrieve_snippets(user_question, top_k=top_k_snippets)

    # retrieve candidate templates from vector store
    candidate_templates = get_candidate_template(user_question, top_k=top_k_templates)

    if news_articles is None and use_news:
      try:
        news_articles = fetch_news(
          user_query=user_question,
          articles_count=news_max_articles,
          days_window=news_days_window,
      )
      except:
        news_articles = None

    # system prompt
    system_prompt = """[SYSTEM]
You are a cautious financial analysis assistant.

Company scope:
- In-scope companies: Nvidia (NVDA), Intel (INTC), TSMC / Taiwan Semiconductor Manufacturing Company (TSM), Samsung / Samsung Electronics.
- If the question is ONLY about companies outside this set:
  - Do NOT analyze them.
  - Use the OUT_OF_SCOPE_COMPANY template and explain that you lack data for those companies.
- If the question mixes in-scope and out-of-scope companies:
  - Clearly say you only have data for the in-scope names.
  - Focus the analysis on in-scope companies and say you cannot properly analyze the others.

Evidence rules:
- Treat snippets as the only source of truth.
- If snippets conflict, say so briefly.
- If snippets are sparse or don’t really answer the question, use the LIMITED_INFORMATION_RESPONSE template.
- Never invent numbers, dates, or details not supported by snippets.
- Never give personalized investment advice (no “you should buy/sell”).
- Use the snippets as evidence and cite them using [SS1], [SS2], etc.
- Use the news articles as supplementary context. Do not cite them.

High-level steps:
1. Identify which companies the user is asking about and whether they are in-scope or out-of-scope. Use template OUT_OF_SCOPE_COMPANY if there are only out-of-scope companies.
2. Review snippets for relevant information about the in-scope companies and the question.
3. Choose exactly ONE template whose “Use when” condition best matches the situation (performance, risk, comparison, limited info, out-of-scope, etc.). Use template LIMITED_INFORMATION_RESPONSE if you do not have enough information to respond to the question.
4. Fill in the template pattern with grounded phrases from the snippets and respond in 2–5 sentences with citations.

Tone:
- Professional, neutral, and concise.
- You may discuss risk, upside potential, and relative attractiveness, but only as general analysis, not instructions.

"""

    # user question block
    user_block = f"[USER QUESTION]\n{user_question}\n"

    # snippets block
    snippets_block = snippet_for_prompt(snippets)

    # templates block
    templates_block = templates_for_prompt(candidate_templates)

    # news block
    news_block = ""
    if news_articles:
      news_block = news_for_prompt(news_articles, max_articles=news_max_articles)
    else:
      news_block = """[RECENT NEWS]
Below are recent news articles related to the company.
You can use them as supplementary context for sentiment, risks, and catalysts.

No relevant news articles within the past 7 days were retrieved."""

    # 7) instructions block
    instructions_block = """
[FALLBACK TEMPLATES — ONLY IF NONE OF THE CANDIDATES APPLY]

[TX] TEMPLATE_ID: LIMITED_INFORMATION_RESPONSE
Use when: Snippets provide little or no relevant information to answer the question confidently.
Pattern: "The available information about Company ___ is limited. The snippets mainly discuss ___ [Sx] and provide little detail on ___ [Sy], so any conclusion would be uncertain."

[TY] TEMPLATE_ID: OUT_OF_SCOPE_COMPANY
Use when: The question is primarily or entirely about companies outside the allowed set.
Pattern: "I do not have sufficient data to analyze the companies mentioned in this question. This system is restricted to discussing Nvidia, Intel, TSMC, and Samsung, and the snippets do not cover the requested company or companies, so I cannot provide a reliable analysis."

[TZ] TEMPLATE_ID: FREEFORM_SUPPORTED
Use when: None of the templates cleanly match, but snippets do contain enough to answer.
Pattern: “Based on the snippets, … [Sx] … [Sy] …”

[RESPONSE FORMAT]
Return ONLY a single-line JSON object (no markdown, no extra text) with EXACTLY these keys:

{
  "template_id": "<ONE template ID you chose (prefer a candidate template; use fallback only if none apply) like MATURE_STABLE_INCOME>",
  "answer": "<2–5 sentences answering the user, grounded in snippets, with citations like [S1], [S2]>",
  "used_snippets": "<The snippets used in your answer like ["S1","S3"], if you used none return []>"
}
"""

    # assemble prompt
    blocks = [
        system_prompt,
        user_block,
        snippets_block,
    ]

    if news_block:
      blocks.append(news_block)

    blocks.extend([
        templates_block,
        instructions_block,
    ])


    full_prompt = "\n\n".join(blocks)

    return {
        "prompt": full_prompt,
        "snippets": snippets,
        "templates": templates_block,
        "news_articles": news_articles,
    }

In [ ]:
test_qu = "Compare Qualcomm to TSMC"
resultu = updated_build_llm_prompt(test_qu)

print(resultu["prompt"])

[SYSTEM]
You are a cautious financial analysis assistant.

Company scope:
- In-scope companies: Nvidia (NVDA), Intel (INTC), TSMC / Taiwan Semiconductor Manufacturing Company (TSM), Samsung / Samsung Electronics.
- If the question is ONLY about companies outside this set:
  - Do NOT analyze them.
  - Use the OUT_OF_SCOPE_COMPANY template and explain that you lack data for those companies.
- If the question mixes in-scope and out-of-scope companies:
  - Clearly say you only have data for the in-scope names.
  - Focus the analysis on in-scope companies and say you cannot properly analyze the others.

Evidence rules:
- Treat snippets as the only source of truth.
- If snippets conflict, say so briefly.
- If snippets are sparse or don’t really answer the question, use the LIMITED_INFORMATION_RESPONSE template.
- Never invent numbers, dates, or details not supported by snippets.
- Never give personalized investment advice (no “you should buy/sell”).
- Use the snippets as evidence and cite th

In [ ]:
def sanitize_for_excel(s: str) -> str:
    s = "" if s is None else str(s)
    # Remove any ASCII control chars except tab/newline/carriage return
    s = "".join(ch for ch in s if (ord(ch) >= 32) or ch in "\t\n\r")
    # Excel cell limit
    return s[:32767]

In [ ]:
from openpyxl import load_workbook

def fill_prompts_in_excel(
    excel_path: str,
    out_path: str | None = None,
    sheet_name: str | None = None,
    question_col: str = "Question",
    prompt_col: str = "Prompt",
    use_news: bool = False,
    overwrite_existing: bool = False,
) -> str:
    """
    Reads `question_col` from an Excel sheet, generates a prompt for each question
    via `updated_build_llm_prompt`, and writes it into `prompt_col`.

    Requires: `updated_build_llm_prompt(question: str, use_news: bool) -> dict`
              returning a dict with key "prompt".
    """
    wb = load_workbook(excel_path)
    ws = wb[sheet_name] if sheet_name else wb[wb.sheetnames[0]]

    # Map headers from the first row -> column index
    headers = {}
    for idx, cell in enumerate(ws[1], start=1):
        if cell.value is not None:
            headers[str(cell.value).strip()] = idx

    if question_col not in headers:
        raise KeyError(f"Could not find column '{question_col}'. Found: {list(headers.keys())}")

    q_idx = headers[question_col]

    # If Prompt column doesn't exist, create it as a new last column
    if prompt_col in headers:
        p_idx = headers[prompt_col]
    else:
        p_idx = ws.max_column + 1
        ws.cell(row=1, column=p_idx).value = prompt_col

    # Fill prompts row-by-row
    for r in range(2, ws.max_row + 1):
        q_val = ws.cell(row=r, column=q_idx).value
        if q_val is None or str(q_val).strip() == "":
            continue

        # Skip if prompt already exists (unless overwrite is requested)
        existing = ws.cell(row=r, column=p_idx).value
        if (existing is not None and str(existing).strip() != "") and not overwrite_existing:
            continue

        question = str(q_val).strip()
        result = updated_build_llm_prompt(question, use_news=use_news)

        if not isinstance(result, dict) or "prompt" not in result:
            raise ValueError(
                f"updated_build_llm_prompt returned unexpected value for row {r}: {result}"
            )

        ws.cell(row=r, column=p_idx).value = sanitize_for_excel(result["prompt"])

    if out_path is None:
        out_path = excel_path.replace(".xlsx", "_with_prompts.xlsx")

    wb.save(out_path)
    return out_path


# --- Usage ---
in_file = "aml_tuning_dataset_two.xlsx"  # adjust path if needed
out_file = fill_prompts_in_excel(
    excel_path=in_file,
    use_news=True,
    overwrite_existing=True,  # set False if you want to preserve any existing prompts
    question_col = "User Query",
    prompt_col = "Prompt"
)

print("Wrote prompts to:", out_file)

Wrote prompts to: aml_tuning_dataset_two_with_prompts.xlsx
